In [30]:
def load(start_idx, end_idx, num_rows, fp_csv, fp_txt):
    if os.path.exists(fp_csv):
        df = pd.read_csv(fp_csv).drop('Unnamed: 0', axis = 1)
        df = df.iloc[start_idx:end_idx,:]
        return df
    
    else:
        train_text = open(fp_txt,"r")
        text_list = train_text.readlines()
        num_rows=num_rows ##########
        df = pd.DataFrame(index=np.arange(num_rows),columns=['label','doc1_title','doc2_title','doc1_body','doc2_body'])
        i=0
        for text in text_list:
            if i>=start_idx and i<=end_idx:
                split_text = text.split('|')
                df['label'][i-start_idx] = split_text[0]
                df['doc1_title'][i-start_idx] = split_text[1]
                df['doc2_title'][i-start_idx] = split_text[2]
                df['doc1_body'][i-start_idx] = split_text[3]
                df['doc2_body'][i-start_idx] = split_text[4]
        i+=1
        return df


[[1, 1], [2, 1], [3, 1], [4, 1]]

In [26]:
len(embed_list[0])

136

In [11]:
with open('embed_2.pkl', 'rb') as f:
    embed_list = pickle.load(f)

In [3]:
import torch 
import utils
import numpy as np


df = utils.load(200,"data/csv/100_train.csv","data/text/100_train.txt")

df = df.iloc[:2,:]

ex_tokens = [str(n)+"." for n in np.arange(50)] # tokens to be excluded
ex_tokens+= [str(n)+". " for n in np.arange(50)]
ex_tokens+= ['S.','S. ']

split_sents = utils.split_into_sentences(df['doc1_body'])

In [1]:
import roberta 



Using cache found in C:\Users\yash/.cache\torch\hub\pytorch_fairseq_master


In [ ]:
len(embd_list)

In [65]:
#function to split text to sentences

#input: text, list of tokens to be excluded
#output: list of sentences

import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

ex_tokens = [str(n)+"." for n in np.arange(50)] # tokens to be excluded
ex_tokens+= [str(n)+". " for n in np.arange(50)]
ex_tokens+=['S.','S. ']

def split_into_sentences(text, ex_tokens):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    if "No." in text: text = text.replace("No.","No<prd>")
    if "Rs." in text: text = text.replace("Rs.","Rs<prd>")
    if "s." in text: text = text.replace("s.","s<prd>")
    if "S." in text: text = text.replace("S.","S<prd>")
    if "cl." in text: text = text.replace("cl.","cl<prd>")
    if "Will." in text: text = text.replace("Will.","Will<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    sentences = split_into_sentences(df['doc1_body'][0])
    split_sentences  = [s for s in sentences if s not in ex_tokens]
    return split_sentences



In [1]:
train_text = open("data/text_data/100_train.txt","r")
text_list = train_text.readlines()


In [2]:
import pandas as pd
import numpy as np 

num_rows=200 ##########
df = pd.DataFrame(index=np.arange(num_rows),columns=['label','doc1_title','doc2_title','doc1_body','doc2_body'])
i=0
for text in text_list:
    split_text = text.split('|')
    df['label'][i] = split_text[0]
    df['doc1_title'][i] = split_text[1]
    df['doc2_title'][i] = split_text[2]
    df['doc1_body'][i] = split_text[3]
    df['doc2_body'][i] = split_text[4]
    i+=1

In [5]:
df.to_csv('train.csv')